In [1]:
import pandas as pd
import tqdm
import rasterio
from rasterio.transform import xy
from rasterio.crs import CRS
import pyproj

# from data_utils import plot_tif_rgb, get_tif_lonlat

def get_lonlat_googlemap_link(lonlat, zoom=14):
    """
    Generate a Google Maps URL pointing to the satellite view of a given latitude and longitude.

    Parameters:
    lonlat (tuple): Tuple of longitude and latitude coordinates.
    zoom (int): Zoom level for the map. Default is 15.

    Returns:
    str: URL string to the Google Maps satellite view.
    """
    lon, lat = lonlat
    base_url = "https://www.google.com/maps"
    # Compose the URL with parameters for latitude, longitude, and satellite view
    url = f"{base_url}/@{lat},{lon},{zoom}z/data=!3m1!1e3"
    return url

def get_tif_lonlat(tif_path, debug=False):
    with rasterio.open(tif_path) as tif:
        # get the affine transformation matrix
        transform = tif.transform
        # get the image width and height
        width = tif.width
        height = tif.height
        # get the coordinate reference system
        crs = tif.crs

        # compute the pixel coordinates of the top-left and bottom-right corners
        top_left_pixel = (0, 0)
        bottom_right_pixel = (width, height)

        # transform the pixel coordinates to image coordinates
        top_left_coords = xy(transform, *top_left_pixel, offset="ul")
        bottom_right_coords = xy(transform, *bottom_right_pixel, offset="lr")

        if debug:
            print(f"Top left coordinates (image CRS): {top_left_coords}")
            print(f"Bottom right coordinates (image CRS): {bottom_right_coords}")

        # transform the image coordinates to longitude and latitude
        if crs.is_geographic:
            top_left_lonlat = top_left_coords
            bottom_right_lonlat = bottom_right_coords
        else:
            transformer = pyproj.Transformer.from_crs(
                crs, CRS.from_epsg(4326), always_xy=True
            )
            top_left_lonlat = transformer.transform(*top_left_coords)
            bottom_right_lonlat = transformer.transform(*bottom_right_coords)
        if debug:
            print(f"Top left coordinates (longitude, latitude): {top_left_lonlat}")
            print(
                f"Bottom right coordinates (longitude, latitude): {bottom_right_lonlat}"
            )
            center_lonlat = (
                (top_left_lonlat[0] + bottom_right_lonlat[0]) / 2,
                (top_left_lonlat[1] + bottom_right_lonlat[1]) / 2,
            )
            print(f"Center coordinates (longitude, latitude): {center_lonlat}")
            googlemap_link = get_lonlat_googlemap_link(center_lonlat)
            print(f"Google Map (for validation): {googlemap_link}")

    return top_left_lonlat, bottom_right_lonlat

In [2]:
# get all chips
chipname_file = "./all_data.txt"
with open(chipname_file, "r") as file:
    all_chips = file.readlines()
    all_chips = [file.strip() for file in all_chips]

print(
    f"Found {len(all_chips)} chips in {chipname_file}\n"
    f"Top 5 chips: {all_chips[:5]}"
)

# get all chip lon lat
chip_lonlat = {}
iterator = tqdm.tqdm(all_chips)
iterator.set_description("Getting chips long lat")
for chip in iterator:
    mask_file = f"./all_chips/{chip}.mask.tif"
    chip_lonlat[chip] = get_tif_lonlat(mask_file, debug=False)

# save to csv
pd.DataFrame(chip_lonlat).T.reset_index().rename(
    columns={0: "topleft", 1: "bottomright", "index": "chip"}
).to_csv("chip_lonlat_map.csv", index=False)
print("Saved to chip_lonlat_map.csv")

# save the dict to file
import pickle
with open("chip_lonlat_map.pkl", "wb") as file:
    pickle.dump(chip_lonlat, file)
print("Saved to chip_lonlat_map.pkl")

Found 3854 chips in ./all_data.txt
Top 5 chips: ['chip_257_266', 'chip_328_501', 'chip_171_477', 'chip_236_281', 'chip_134_482']


Getting chips long lat: 100%|██████████| 3854/3854 [02:45<00:00, 23.25it/s]


Saved to chip_lonlat_map.csv
Saved to chip_lonlat_map.pkl
